In [1]:
import findspark
import warnings
findspark.init()

from pyspark.sql import  SparkSession, Row
from pyspark.sql.functions import \
	lit, col, when, expr, countDistinct,\
	monotonically_increasing_id, desc
spark = SparkSession.builder.appName('test_preprocesdid')\
    .config('spark.master', 'local[4]')\
    .config('spark.executor.memory', '1g')\
    .config("spark.sql.shuffle.partitions", 1)\
    .config('spark.driver.memory','1g')\
    .getOrCreate()

from csdid.ATTgt import ATTgt

In [2]:
%%time
import os
name = r'D:\Workflow\work\csdid_r\R\5g10t.csv'
# os.path.exists(name)

yname, gname, idname, tname = 'Y', 'G', 'id', 'period'
control_group = ['nevertreated', 'notyettreated']
anticipation = 0
panel = True
allow_unbalanced_panel = True
weights_name = None

clustervar = None
xfmla = None

data = spark.read.csv(name, header=True, inferSchema=True)
# data.show(5)
a = ATTgt(data=data, tname=tname, gname=gname, yname=yname, idname=idname)

CPU times: total: 31.2 ms
Wall time: 9.64 s


In [9]:
import numpy as np
base_period='varying'
yname = a.yname
tname = a.tname
idname = a.idname
gname = a.gname
xfmla = a.xfmla
data = a.new_data
weights_name = a.weights_name
panel = a.panel
control_group= a.control_group
anticipation = a.anticipation

n = a.n
nT = a.nT
nG = a.nG

tlist = a.tlist.collect()
glist = a.glist.collect()

tlist = [x[tname] for x in tlist]
glist = [x[gname] for x in glist]
tlist, glist = map(np.array, [tlist, glist])
print(tlist, glist)

tlist_len, tfac = len(tlist), 0

if base_period != 'universal':
	tlist_len = tlist_len - 1
	tfac = 1

never_treated = control_group == 'nevertreated'
if never_treated:
	data = data.withColumn('C', when(col(gname) == 0, 1).otherwise(0))

data = data.withColumn('y_main', col(yname))


[ 1  2  3  4  5  6  7  8  9 10] [2 3 4 5 6]


In [11]:
for _, g in enumerate(glist):
	print(g)
g = glist[0]
t_i = 0

2
3
4
5
6


In [23]:
g, tlist[t_i + tfac]
# tlist[]


(2, 2)

In [25]:
# for _, g in enumerate(glist):
data = data.withColumn(
'G_m', when(
	col(gname) == g, 1
).otherwise(0)
)
	# for t_i in range(tlist_len):
pret = t_i
tn = tlist[t_i + tfac]

if base_period == 'universal' or g < tn:
	try:
		pret = np.where(tlist + anticipation < g)[0][-1]
	except:
		raise f"There are no pre-treatment periods for the group first treated at {g}\nUnits from this group are dropped"

if base_period == 'universal':
	if pret == tn:
		add_att_data()
	
if not never_treated:
	# base
	# n1 = data[gname] == 0
	# n2 = (data[gname] > (tlist[np.max([t_i, pret]) + tfac]) + anticipation)
	# n3 = np.where(data[gname] != glist[g], True, False)
	# row_eval = n1 | n2 & n3

	data = data.withColumn('n1', col(gname) == 0)
	value_n2 = (tlist[np.max([t_i, pret]) + tfac]) + anticipation
	data = data.withColumn(
	'n2', col(gname) > value_n2
	).withColumn(
	'n3', col(gname) != glist[g]
	)
	data = data.withColumn(
	'row_eval', (col('n1') | col('n2') & col('n3'))
	).withColumn('C', col('row_eval'))

In [6]:
import numpy as np
a = np.ones((12, 1))
a[:, 0] == np.ones(12)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

: 